In [3]:
import numpy as np
import math
import pandas as pd
#import pr_z_v6 as pr_z

import importlib
#importlib.reload(pr_z)
#import pyrestoolbox as rtb
#importlib.reload(rtb)

import gerg2 as gerg
importlib.reload(gerg)

from tabulate import tabulate as tabulate

In [36]:
base_chr = 'RealGas'
fileloc = 'HC_BIPS_v8_GERG'

mwAir = 28.97

mw_hcs = [16.043, 25.977, 33.864, 45.08779652]
names=['lean_gas','leanmid_gas', 'mid_gas', 'rich_gas' ]
xis = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0.831940685, 0.045074732, 0.024596917, 0.00670825, 0.012828057, 0.007061316, 0.00670825, 0.010945039, 0.008189118, 0.006202202, 0.004697369, 0.035048066],
       [0.663296918, 0.104190445, 0.065873145, 0.011576716, 0.021359857, 0.008315669, 0.00766346, 0.017935757, 0.027555845, 0.011739768, 0.00978314, 0.050709278],
       [0.336259298, 0.159219833, 0.160102696, 0.034431669, 0.087027998, 0.039688258, 0.042935571, 0.050343505, 0.03731366, 0.023674944, 0.005987234, 0.023015333]]

degfs = [60, 90, 120, 150, 180, 220, 250, 280]
psias = [1500, 2000, 2500, 5000, 7500, 10000, 15000]
hc_fractions = [0.8, 0.6, 0.4, 0.2]

compnames = ['CO2', 'H2S', 'N2', 'H2', 'Gas']
comp_dic = {0: 'CO2', 1: 'H2S', 2: 'N2', 3: 'H2'}
all_degfs, all_sgs, all_psias, all_zs, all_comps = [[] for x in range(5)]

for j in range(len(xis)):
    mw_hc = mw_hcs[j]
    name = names[j]
    xi = xis[j]
    for i in range(len(compnames)-1):   
        out_str = 'DEFINE MW '+str(mw_hc)+'\n'
        for frac in hc_fractions:
            if i == 0:
                co2, h2s, n2, h2 = 1-frac, 0, 0, 0
            if i == 1:
                co2, h2s, n2, h2 = 0, 1-frac, 0, 0
            if i == 2:
                co2, h2s, n2, h2 = 0, 0,  1-frac, 0
            if i == 3:
                co2, h2s, n2, h2 = 0, 0,  0, 1-frac
            comps = [co2, h2s, n2, h2, frac]
            
            sg = (frac * mw_hc + co2 * 44.01 + h2s * 34.082 + n2 * 28.014 + h2 * 2.016) / mwAir
            
            for degf in degfs:
                zs = []
                for psia in psias:
                    all_degfs.append(degf)
                    all_sgs.append(sg)
                    all_psias.append(psia)
                    all_comps.append([co2, h2s, n2, h2])
                    z = gerg.gerg_z2(psia, degf, xi, co2, h2s, n2, h2, iFlag = 1)
                    all_zs.append(z)
                    zs.append(z)
                    
                out_str += 'DEFINE degR '+str(degf+459.67)+'\n\n'
                out_str += 'CHARACTERIZATION "CHR_'+str(degf)+'_'+str(frac)+'_'+comp_dic[i]+'" BASE "'+base_chr+'"\n'
                header = ['COMPONENT', 'MW', 'TC,R', 'PC,PSIA', 'AF', 'VTRAN', 'A', 'B', 'AMOD', 'BMOD']
                rows = []
                for c in range(len(compnames)):
                    rows.append([compnames[c], '', '', '', '', '', '', '', '', ''])
                rows[-1][1] = round(mw_hc, 4)
    
                out_str += tabulate(rows, header)
                out_str += '\nEND\n\n'
                
                out_str += 'DEFINE degR_Slope CO2_degR_Slope, DEFINE MW_Slope CO2_MW_Slope, DEFINE Intcpt CO2_Intcpt, DEFINE Inert CO2\n'
                out_str += 'INCLUDE bips_calc.inc\n\n'
                
                out_str += 'DEFINE degR_Slope H2S_degR_Slope, DEFINE MW_Slope H2S_MW_Slope, DEFINE Intcpt H2S_Intcpt, DEFINE Inert H2S\n'
                out_str += 'INCLUDE bips_calc.inc\n\n'
                
                out_str += 'DEFINE degR_Slope N2_degR_Slope, DEFINE MW_Slope N2_MW_Slope, DEFINE Intcpt N2_Intcpt, DEFINE Inert N2\n'
                out_str += 'INCLUDE bips_calc.inc\n\n'
                
                out_str += 'DEFINE degR_Slope H2_degR_Slope, DEFINE MW_Slope H2_MW_Slope, DEFINE Intcpt H2_Intcpt, DEFINE Inert H2\n'
                out_str += 'INCLUDE bips_calc.inc\n\n'
                temp = degf
            
                ## Inert:Inert BIP Pairs                                                                                                   
                #slopes = [[-0.000879210909090909, -0.00130965, 0.00118182785714286, -0.00150498, 0.00461018571428571, 0.000235450357142857],                                
                #          [0.00009085, 0.000541365245623941, 0.00135899363636364, 0.00103136527385658, 0.00112685210526316, 0.000149872727272727]]                                    
                #intcpts = [[0.239517381818182, -0.085784, 0.203866711904762, 0.85241, 0.400074571428572, 0.204373053571429],                                   
                #           [0.0880919666666667, -0.247960272896669, 0.172206318181818, 0.601450271597967, 0.822955005263158, 0.220908090909091]]                               
                #evals = [np.max, np.max, np.max, np.max, np.min, np.min]                                                                  
                #pairs = ['H2S with CO2', 'CO2 with N2', 'H2S WITH N2', 'CO2 with H2', 'H2S with H2', 'N2 with H2']                       
                #inert_bips = [evals[i]([slopes[0][i]*temp + intcpts[0][i], slopes[1][i]*temp + intcpts[1][i]]) for i in range(len(pairs))]
                #
                #for p, pair in enumerate(pairs):                                                                                          
                #    out_str += 'REPLACE BIP of '+pair+' by '+str(inert_bips[p])+'\n'   
                    
                #BIP_HC = max(0.000160492*temp + 0.056787492, 1/(0.040933216*temp + 2.454878276)) 
                #BIP_CN = 1/(-0.061940854*temp + -0.494913095)                     
                #BIP_HN = -0.836144851 + (temp * 2.80825315) / (348.2693873 + temp)

                #out_str += 'REPLACE BIP of H2S with CO2 by '+str(BIP_HC)+'\n'
                #out_str += 'REPLACE BIP of CO2 with N2 by '+str(BIP_CN)+'\n'
                #out_str += 'REPLACE BIP of H2S with N2 by '+str(BIP_HN)+'\n\n'
                
                out_str += 'REPLACE BIP of CO2 with H2S by CO2_H2S\n'
                out_str += 'REPLACE BIP of CO2 with N2 by CO2_N2\n'
                out_str += 'REPLACE BIP of CO2 with H2 by CO2_H2\n'
                out_str += 'REPLACE BIP of H2S with N2 by H2S_N2\n'
                out_str += 'REPLACE BIP of H2S with H2 by H2S_H2\n'
                out_str += 'REPLACE BIP of N2 with H2 by N2_H2\n\n'
            
                
                out_str += '\nINCLUDE '+name+'_init.inc\n\n'   

                out_str += 'MIX FEED\n'
                
                rows = []
                for c in range(len(compnames)):
                    rows.append([compnames[c], comps[c]])
                out_str += tabulate(rows)
                out_str += '\nEND\n\n'
            
                out_str += 'CCE ID "'+str(degf)+'_'+str(frac)+'_'+comp_dic[i]+'"\n'
    
                exp_df = pd.DataFrame()
                
                exp_df['TEMP (F)']= [degf for z in zs]
                exp_df['PRES (psia)']= psias
                exp_df['Z']= zs
    
                out_str += tabulate(exp_df, headers='keys', showindex='never', numalign='left')
                out_str += '\nEND\n\n'
        
        with open('.\\'+fileloc+'\\'+name+comp_dic[i]+'.inc', 'w') as file:
            file.write(out_str)


exp_df = pd.DataFrame()
exp_df['psia']= all_psias
exp_df['Z']= all_zs
exp_df['degf']= all_degfs
exp_df['sg']= all_sgs
exp_df['co2']= [comp[0] for comp in all_comps]
exp_df['h2s']= [comp[1] for comp in all_comps]
exp_df['n2']= [comp[2] for comp in all_comps]
exp_df['h2']= [comp[3] for comp in all_comps]
exp_df['Sample']= ['GERG' for z in all_zs]
exp_df['Source']= ['GERG' for z in all_zs]  

            


In [37]:
exp_df.to_excel('GERG_Synthetic_Data.xlsx', index=False)

## Make Inert:Inert combinations

In [34]:
base_chr = 'RealGas'
fileloc = 'HC_BIPS_v8_GERG'

mwAir = 28.97
xi = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
degfs = [60, 90, 120, 150, 180, 220, 250, 280]
psias = [1500, 2000, 2500, 5000, 7500, 10000, 15000]

compnames = ['CO2', 'H2S', 'N2', 'H2']
all_degfs, all_sgs, all_psias, all_zs, all_comps = [[] for x in range(5)]

for j in range(len(compnames)-1):
    for i in range(j+1, len(compnames)): 
        yi = 4*[0]
        yi[i] = 0.5
        yi[j] = 0.5
        sg = (frac * mw_hc + yi[0] * 44.01 + yi[1] * 34.082 + yi[2] * 28.014 + yi[3] * 2.016) / mwAir
        out_str = 'TITLE "50:50 '+compnames[j]+' with '+compnames[i]+'"\n\n'    
        for degf in degfs:                                                                                                                                                         
            zs = []                                                                                                                                                                
            for psia in psias:                                                                                                                                                     
                all_degfs.append(degf)                                                                                                                                             
                all_sgs.append(sg)                                                                                                                                                 
                all_psias.append(psia)                                                                                                                                             
                all_comps.append([co2, h2s, n2, h2])                                                                                                                               
                z = gerg.gerg_z2(psia, degf, xi, yi[0], yi[1], yi[2], yi[3], iFlag = 1)                                                                                                      
                all_zs.append(z)                                                                                                                                                   
                zs.append(z)                                                                                                                                                       
                                                                                                                                                                                   
            out_str += 'DEFINE degR '+str(degf+459.67)+'\n\n'                                                                                                                      
            out_str += 'CHARACTERIZATION "CHR_'+str(degf)+'_'+compnames[j]+'_'+compnames[i]+'" BASE "'+base_chr+'"\n'                                                                  
            header = ['COMPONENT', 'MW', 'TC,R', 'PC,PSIA', 'AF', 'VTRAN', 'A', 'B', 'AMOD', 'BMOD']                                                                               
            rows = []                                                                                                                                                              
            for c in range(len(compnames)):                                                                                                                                        
                rows.append([compnames[c], '', '', '', '', '', '', '', '', ''])                                                                                                    
                                                                                                                                        
                                                                                                                                                                                   
            out_str += tabulate(rows, header)                                                                                                                                      
            out_str += '\nEND\n\n'                                                                                                                                                                                                                                                
                                                                                                                                                                                   
            out_str += 'REPLACE BIP of CO2 with H2S by CO2_H2S\n'
            out_str += 'REPLACE BIP of CO2 with N2 by CO2_N2\n'
            out_str += 'REPLACE BIP of CO2 with H2 by CO2_H2\n'
            out_str += 'REPLACE BIP of H2S with N2 by H2S_N2\n'
            out_str += 'REPLACE BIP of H2S with H2 by H2S_H2\n'
            out_str += 'REPLACE BIP of N2 with H2 by N2_H2\n\n'                                                                                                                                                                                                                                                                                                                                                                                                                     
                                                                                                                                                                                   
            out_str += 'MIX FEED\n'                                                                                                                                                
                                                                                                                                                                                   
            rows = []                                                                                                                                                              
            for c in range(len(compnames)):                                                                                                                                        
                rows.append([compnames[c], yi[c]])                                                                                                                              
            out_str += tabulate(rows)                                                                                                                                              
            out_str += '\nEND\n\n'                                                                                                                                                 
                                                                                                                                                                                   
            out_str += 'CCE ID "'+compnames[j]+'_'+compnames[i]+'_'+str(degf)+'"\n'                                                                                                    
                                                                                                                                                                                   
            exp_df = pd.DataFrame()                                                                                                                                                
                                                                                                                                                                                   
            exp_df['TEMP (F)']= [degf for z in zs]                                                                                                                                 
            exp_df['PRES (psia)']= psias                                                                                                                                           
            exp_df['Z']= zs  
            exp_df['Wt']= '?inert_wts?'
                                                                                                                                                                                   
            out_str += tabulate(exp_df, headers='keys', showindex='never', numalign='left')                                                                                        
            out_str += '\nEND\n\n'                                                                                                                                                 
                                                                                                                                                                                       
        with open('.\\'+fileloc+'\\'+compnames[j]+'_'+compnames[i]+'.inc', 'w') as file:                                                                                                            
            file.write(out_str)   
            print(compnames[j]+'_'+compnames[i]+'.inc')
                                                                                                                                                                                       



CO2_H2S.inc
CO2_N2.inc
CO2_H2.inc
H2S_N2.inc
H2S_H2.inc
N2_H2.inc


## And make data for pure paraffins

In [55]:
coefic_pc = [0.00514195, 1.50107866]
coefic_tc = [0.00244823, 0.30153678]
base_chr = 'RealGas'
fileloc = 'GERG_Pure'
hc_names = ['C1', 'C2', 'C3', 'I-C4', 'N-C4', 'I-C5', 'N-C5', 'N-C6', 'BENZENE', 'N-C7', 'C-C7', 'N-C8', 'N-C9', 'N-C10']

names=['lean_gas','leanmid_gas', 'mid_gas', 'leanmid_gas_aromat','mid_gas_aromat']#, 'rich_gas' ]
xis = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0.831940685, 0.045074732, 0.024596917, 0.00670825, 0.012828057, 0.007061316, 0.00670825, 0.010945039, 0, 0.008189118, 0, 0.006202202, 0.004697369, 0.035048066],
       [0.663296918, 0.104190445, 0.065873145, 0.011576716, 0.021359857, 0.008315669, 0.00766346, 0.017935757, 0, 0.027555845, 0, 0.011739768, 0.00978314, 0.050709278],
       [0.831940685, 0.045074732, 0.024596917, 0.00670825, 0.012828057, 0.007061316, 0.00670825, 0, 0.010945039, 0, 0.008189118, 0.006202202, 0.004697369, 0.035048066],
       [0.663296918, 0.104190445, 0.065873145, 0.011576716, 0.021359857, 0.008315669, 0.00766346, 0, 0.017935757, 0, 0.027555845, 0.011739768, 0.00978314, 0.050709278]]
# [0.336259298, 0.159219833, 0.160102696, 0.034431669, 0.087027998, 0.039688258, 0.042935571, 0.050343505, 0, 0.03731366, 0, 0.023674944, 0.005987234, 0.023015333] # Rich
degfs = [60, 90, 120, 150, 180, 220, 250, 280]
psias = [1500, 2000, 2500, 5000, 7500, 10000, 15000]

pressures = {} # Set pressure ranges to fit over to avoid issues with condensate
for degf in degfs:
    pressures['lean_gas'+str(degf)] = psias
    pressures['leanmid_gas'+str(degf)] = psias[3:]
    pressures['mid_gas'+str(degf)] = psias[3:]


hc_fractions = [1.0]
all_degfs, all_sgs, all_psias, all_zs, all_comps = [[] for x in range(5)]

for j in range(len(names)):
    name = names[j]
    xi = xis[j]
    #xi = [x/sum(xi) for x in xi]
    for i in range(3):   
        out_str = 'TITLE '+name+'\n'
        for frac in hc_fractions:
            
            out_str += 'MIX FEED\n'
                
            rows = []
            for c in range(len(hc_names)):
                rows.append([hc_names[c], xi[c]])
            out_str += tabulate(rows)
            out_str += '\nEND\n\n'
                
            for degf in degfs:
                
                zs = []
                psia_name = name+str(degf)
                psia_name = psia_name.replace('_aromat','')
                for psia in pressures[psia_name]:
                    all_degfs.append(degf)
                    all_sgs.append(sg)
                    all_psias.append(psia)
                    all_comps.append([co2, h2s, n2])
                    if 'aromat' in name:
                        z = ''
                    else:
                        xj = xi[:8]+[xi[9]]+xi[11:]
                        z = gerg.gerg_z2(psia, degf, xj, 0, 0, 0, iFlag = 1)
                    all_zs.append(z)
                    zs.append(z)
                
                
            
                out_str += 'CCE ID "'+str(degf)+'_'+name+'"\n'
    
                exp_df = pd.DataFrame()
                
                exp_df['TEMP (F)']= [degf for z in zs]
                exp_df['PRES (psia)']= pressures[psia_name]
                exp_df['Z']= zs
                exp_df['LMF']= ['' for z in zs]
    
                out_str += tabulate(exp_df, headers='keys', showindex='never', numalign='left')
                out_str += '\nEND\n\n'
        
        with open('.\\'+fileloc+'\\'+name+'.inc', 'w') as file:
            file.write(out_str)


exp_df = pd.DataFrame()
exp_df['psia']= all_psias
exp_df['Z']= all_zs
exp_df['degf']= all_degfs
exp_df['sg']= all_sgs
exp_df['co2']= [comp[0] for comp in all_comps]
exp_df['h2s']= [comp[1] for comp in all_comps]
exp_df['n2']= [comp[2] for comp in all_comps]
exp_df['Sample']= ['GERG' for z in all_zs]
exp_df['Source']= ['GERG' for z in all_zs]          
            


In [51]:
psia_name.replace('aromat','')

'lean_mid_60'

In [41]:
xi

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]